## Libraries and variables

In [3]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

data_path = '../01_Data/'
file_name_cleaned_data = '3-1_psp-data_cleaned.csv'
file_path_cleaned_data = data_path + file_name_cleaned_data

## Data load

In [4]:
df = pd.read_csv(file_path_cleaned_data, index_col=0)
df['tmsp'] = pd.to_datetime(df['tmsp'])

## Preprocessing
https://scikit-learn.org/stable/modules/preprocessing.html \
https://scikit-learn.org/stable/api/sklearn.preprocessing.html

### Data enrichment

In [5]:
# Add fee column
def calculate_fee(row):
    if row['PSP'] == 'Moneycard':
        return 5.0 if row['success'] == 1 else 2.0
    elif row['PSP'] == 'Goldcard':
        return 10.0 if row['success'] == 1 else 5.0
    elif row['PSP'] == 'UK_Card':
        return 3.0 if row['success'] == 1 else 1.0
    elif row['PSP'] == 'Simplecard':
        return 1.0 if row['success'] == 1 else 0.5
    else:
        return 0
    
df['fee'] = df.apply(calculate_fee, axis=1)

In [6]:
# Add transaction hour
df['transaction_hour'] = df['tmsp'].dt.hour

In [7]:
# Add purchase index
# A purchase is defined as follows: previous transaction timestamp is <= 60 seconds and country, amount, card and 3D_secured are equal
def assign_purchase(df):
    purchases = []
    purchase_num = 0
    last_transaction_time = None
    
    for index, row in df.iterrows():
        if last_transaction_time is None or \
            (row['tmsp'] - last_transaction_time).total_seconds() >= 60 or \
            (row['country'], row['amount'], row['card'], row['3D_secured']) != last_transaction:
            
            purchase_num += 1
        
        purchases.append(purchase_num)
        last_transaction_time = row['tmsp']
        last_transaction = (row['country'], row['amount'], row['card'], row['3D_secured'])
    
    return purchases


df['purchase'] = assign_purchase(df)

In [8]:
# Add amount buckets separated by 50 Euros
df['amount_bucket'] = pd.cut(df['amount'], bins=range(0, max(df['amount'])+51, 50))

### Feature encoding
One-Hot-Encoding, Ordinal-Encoding, Target-Encoding

In [9]:
# One-Hot-Encodig for colums 'card', 'PSP' and 'country'
# Decision in favour of One-Hot-Encodign due to fewer categorical values and no order in the categorical values
encoder = OneHotEncoder()

encoded_data = encoder.fit_transform(df[['card', 'PSP', 'country']])
encoded_df = pd.DataFrame(encoded_data.toarray(), columns=encoder.get_feature_names_out(['card', 'PSP', 'country']))

df = pd.concat([df, encoded_df], axis=1)

### Scaling and normalization
- StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler
- Normalizer (L1, L2, Max-Norm)

In [10]:
min_max_scaler = MinMaxScaler()
df[["amount_sc", "transaction_hour_sc"]] = min_max_scaler.fit_transform(df[["amount", "transaction_hour"]])
df.head()

,tmsp,country,amount,success,PSP,3D_secured,card,fee,transaction_hour,purchase,...,card_Visa,PSP_Goldcard,PSP_Moneycard,PSP_Simplecard,PSP_UK_Card,country_Austria,country_Germany,country_Switzerland,amount_sc,transaction_hour_sc
0,2019-01-01 00:01:11,Germany,89,0,UK_Card,0,Visa,1.0,0,1,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.133013,0.0
1,2019-01-01 00:01:17,Germany,89,1,UK_Card,0,Visa,3.0,0,1,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.133013,0.0
2,2019-01-01 00:02:49,Germany,238,0,UK_Card,1,Diners,1.0,0,2,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.371795,0.0
3,2019-01-01 00:03:13,Germany,238,1,UK_Card,1,Diners,3.0,0,2,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.371795,0.0
4,2019-01-01 00:04:33,Austria,124,0,Simplecard,0,Diners,0.5,0,3,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.189103,0.0


## Classification - Save preprocessed data

In [11]:
df.to_csv(data_path + '3-2_psp-data_prepro_class.csv')

## Regression - Save preprocessed data

In [12]:
# Create data frame grouped by purchase
df_regr = df.groupby(['country', 'amount', 'amount_sc', '3D_secured', 'card', 'country_Austria', 'country_Germany', 'country_Switzerland', 'card_Diners', 'card_Master', 'card_Visa', 'purchase', 'PSP', "PSP_Goldcard", "PSP_Moneycard", "PSP_Simplecard", "PSP_UK_Card"]).agg({
    'fee': 'sum',           # Sum of fees
    'success': 'max',       # Max of success (1 if overall status was successful)
    'country': 'size'       # Count of transactions for each purchase (country used as dummy column)
}).rename(columns={'country': 'transaction_count', 'success':'success_max'}).reset_index()

# purchase_df_success = purchase_df[purchase_df['success_max']==1]
# purchase_df_failure = purchase_df[purchase_df['success_max']==0]

df_regr.to_csv(data_path + '3-2_psp-data_prepro_regr.csv')

In [13]:
df_regr

,country,amount,amount_sc,3D_secured,card,country_Austria,country_Germany,country_Switzerland,card_Diners,card_Master,card_Visa,purchase,PSP,PSP_Goldcard,PSP_Moneycard,PSP_Simplecard,PSP_UK_Card,fee,success_max,transaction_count
0,Austria,6,0.000000,0,Diners,1.0,0.0,0.0,1.0,0.0,0.0,4591,Moneycard,0.0,1.0,0.0,0.0,2.0,0,1
1,Austria,6,0.000000,0,Diners,1.0,0.0,0.0,1.0,0.0,0.0,4591,Simplecard,0.0,0.0,1.0,0.0,0.5,0,1
2,Austria,6,0.000000,0,Diners,1.0,0.0,0.0,1.0,0.0,0.0,18124,Simplecard,0.0,0.0,1.0,0.0,0.5,0,1
3,Austria,6,0.000000,0,Diners,1.0,0.0,0.0,1.0,0.0,0.0,18124,UK_Card,0.0,0.0,0.0,1.0,2.0,0,2
4,Austria,6,0.000000,0,Master,1.0,0.0,0.0,0.0,1.0,0.0,12408,Simplecard,0.0,0.0,1.0,0.0,1.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38217,Switzerland,495,0.783654,1,Master,0.0,0.0,1.0,0.0,1.0,0.0,10027,Simplecard,0.0,0.0,1.0,0.0,0.5,0,1
38218,Switzerland,497,0.786859,1,Master,0.0,0.0,1.0,0.0,1.0,0.0,6249,Moneycard,0.0,1.0,0.0,0.0,5.0,1,1
38219,Switzerland,497,0.786859,1,Master,0.0,0.0,1.0,0.0,1.0,0.0,6249,UK_Card,0.0,0.0,0.0,1.0,1.0,0,1
38220,Switzerland,499,0.790064,0,Master,0.0,0.0,1.0,0.0,1.0,0.0,1158,Simplecard,0.0,0.0,1.0,0.0,0.5,0,1
